In [13]:
# Import libraries
import pandas as pd
import numpy as np
from oandapyV20 import API
from oandapyV20.endpoints.instruments import InstrumentsCandles

# ===== YOUR CREDENTIALS () =====
ACCESS_TOKEN = "32a7ce5d62167ef2dda800802386d48a-496bae9b01506113e0a75dbac8e7a693"  # 

# 1. Function to fetch historical data for one instrument
def fetch_forex_data(instrument="EUR_USD", granularity="M1", count=5000):
    client = API(access_token=ACCESS_TOKEN, environment="practice")

    params = {
        "granularity": granularity,
        "count": count
    }

    r = InstrumentsCandles(instrument=instrument, params=params)

    try:
        client.request(r)
    except Exception as e:
        raise ConnectionError(f"Failed to fetch data for {instrument}: {e}")

    data = []
    for candle in r.response['candles']:
        data.append({
            'datetime': candle['time'],
            'close': float(candle['mid']['c'])
        })

    df = pd.DataFrame(data)
    df['datetime'] = pd.to_datetime(df['datetime'])
    df.set_index('datetime', inplace=True)
    return df[['close']].rename(columns={'close': instrument})

# 2. Define triangles
def get_triangles():
    return [
        ["EUR", "USD", "GBP"],
        ["AUD", "USD", "CAD"],
        ["NZD", "USD", "CHF"],
        ["EUR", "JPY", "USD"]
    ]

# 3. Detect arbitrage opportunity at a single timestamp
def detect_opportunity(triangle, row):
    c1, c2, c3 = triangle
    p1 = f"{c1}_{c2}"
    p2 = f"{c2}_{c3}"
    p3 = f"{c3}_{c1}"

    try:
        # Forward path: c1 -> c2 -> c3 -> c1
        val = 1.0
        val *= row[p1]
        val *= row[p2]
        val *= row[p3]

        profit = val - 1.0
        return profit
    except KeyError:
        return None

# 4. Main backtest function
def run_triangular_arbitrage_backtest(pairs, granularity="H1", count=100):
    dfs = []
    print("📥 Fetching historical data for:")
    for pair in pairs:
        print(f"  - {pair}")
        df = fetch_forex_data(pair, granularity, count)
        dfs.append(df)

    # Merge all into one DataFrame
    merged = pd.concat(dfs, axis=1)
    merged.dropna(inplace=True)
    merged.sort_index(inplace=True)

    print("\n📊 Merged DataFrame Sample:")
    print(merged.head())

    triangles = get_triangles()
    results = []

    print("\n🔍 Running backtest for triangular arbitrage opportunities...\n")
    for idx, row in merged.iterrows():
        found = False
        for triangle in triangles:
            profit = detect_opportunity(triangle, row)
            if profit is not None and profit > 0.0001:
                print(f"✅ Opportunity at {idx} | Triangle: {triangle} | Profit: {profit*100:.4f}%")
                results.append({
                    "timestamp": idx,
                    "triangle": str(triangle),
                    "profit_percent": round(profit * 100, 4)
                })
                found = True
        if not found:
            print(f"❌ No opportunity at {idx}")

    print("\n📈 Summary of Opportunities:")
    result_df = pd.DataFrame(results)
    if not result_df.empty:
        print(result_df.to_string(index=False))
    else:
        print("None found.")
    return result_df

# ===== RUN FULL BACKTEST =====
if __name__ == "__main__":
    # List of forex pairs needed for the triangles
    forex_pairs = [
        "EUR_USD", "GBP_USD", "GBP_CHF",
        "AUD_USD",  "NZD_USD",
        "USD_JPY", "EUR_JPY"
    ]

    # Run the backtest
    arbitrage_results = run_triangular_arbitrage_backtest(
        pairs=forex_pairs,
        granularity="M1",   # You can change to H4, D etc.
        count=5000           # Number of candles to fetch
    )

📥 Fetching historical data for:
  - EUR_USD
  - GBP_USD
  - GBP_CHF
  - AUD_USD
  - NZD_USD
  - USD_JPY
  - EUR_JPY

📊 Merged DataFrame Sample:
                           EUR_USD  GBP_USD  GBP_CHF  AUD_USD  NZD_USD  \
datetime                                                                 
2025-06-10 21:53:00+00:00  1.14262  1.35004  1.11096  0.65242  0.60516   
2025-06-10 21:54:00+00:00  1.14260  1.35006  1.11082  0.65246  0.60516   
2025-06-10 21:55:00+00:00  1.14254  1.35014  1.11092  0.65240  0.60517   
2025-06-10 21:56:00+00:00  1.14258  1.35016  1.11091  0.65244  0.60521   
2025-06-10 21:57:00+00:00  1.14260  1.35018  1.11094  0.65246  0.60521   

                           USD_JPY  EUR_JPY  
datetime                                     
2025-06-10 21:53:00+00:00  144.874  165.535  
2025-06-10 21:54:00+00:00  144.875  165.540  
2025-06-10 21:55:00+00:00  144.868  165.520  
2025-06-10 21:56:00+00:00  144.868  165.511  
2025-06-10 21:57:00+00:00  144.873  165.516  

🔍 Running back

In [31]:
# Import libraries
import pandas as pd
import numpy as np
from oandapyV20 import API
from oandapyV20.endpoints.instruments import InstrumentsCandles

# ===== YOUR CREDENTIALS () =====
ACCESS_TOKEN = "32a7ce5d62167ef2dda800802386d48a-496bae9b01506113e0a75dbac8e7a693"  # Replace with env var in production

# 1. Function to fetch historical data for one instrument
def fetch_forex_data(instrument="EUR_USD", granularity="M1", count=60):  # M1 = Monthly, 60 = 5 years of data
    client = API(access_token=ACCESS_TOKEN, environment="practice")

    params = {
        "granularity": granularity,
        "count": count
    }

    r = InstrumentsCandles(instrument=instrument, params=params)

    try:
        client.request(r)
    except Exception as e:
        raise ConnectionError(f"Failed to fetch data for {instrument}: {e}")

    data = []
    for candle in r.response['candles']:
        data.append({
            'datetime': candle['time'],
            'close': float(candle['mid']['c'])
        })

    df = pd.DataFrame(data)
    df['datetime'] = pd.to_datetime(df['datetime'])
    df.set_index('datetime', inplace=True)
    return df[['close']].rename(columns={'close': instrument})

# 2. Define triangles
def get_triangles():
    return [
        ["EUR", "USD", "GBP"],   # Uses EUR_USD, GBP_USD, GBP_EUR
        ["AUD", "USD", "NZD"],   # Uses AUD_USD, NZD_USD
        ["USD", "JPY", "EUR"],   # Uses USD_JPY, EUR_JPY
    ]

# 3. Detect arbitrage opportunity at a single timestamp
def detect_opportunity(triangle, row):
    c1, c2, c3 = triangle
    p1 = f"{c1}_{c2}"
    p2 = f"{c2}_{c3}"
    p3 = f"{c3}_{c1}"

    try:
        val = 1.0
        val *= row[p1]
        val *= row[p2]
        val *= row[p3]

        profit = val - 1.0
        return profit
    except KeyError:
        return None

# 4. Main backtest function
def run_triangular_arbitrage_backtest(pairs, granularity="M1", count=60):
    dfs = []
    print("📥 Fetching monthly historical data for:")
    for pair in pairs:
        print(f"  - {pair}")
        df = fetch_forex_data(pair, granularity, count)
        dfs.append(df)

    # Merge all into one DataFrame
    merged = pd.concat(dfs, axis=1)
    merged.dropna(inplace=True)
    merged.sort_index(inplace=True)

    print("\n📊 Merged Monthly DataFrame Sample:")
    print(merged.head())

    triangles = get_triangles()
    results = []

    print("\n🔍 Running monthly triangular arbitrage backtest...\n")
    for idx, row in merged.iterrows():
        found = False
        for triangle in triangles:
            profit = detect_opportunity(triangle, row)
            if profit is not None and profit > 0.0001:
                print(f"✅ Opportunity at {idx} | Triangle: {triangle} | Profit: {profit*100:.4f}%")
                results.append({
                    "timestamp": idx,
                    "triangle": str(triangle),
                    "profit_percent": round(profit * 100, 4)
                })
                found = True
        if not found:
            print(f"❌ No opportunity at {idx}")

    print("\n📈 Summary of Monthly Arbitrage Opportunities:")
    result_df = pd.DataFrame(results)
    if not result_df.empty:
        print(result_df.to_string(index=False))
    else:
        print("None found.")
    return result_df

# ===== RUN FULL BACKTEST =====
if __name__ == "__main__":
    # List of forex pairs needed for the triangles
    forex_pairs = [
        "EUR_USD", "GBP_USD", "GBP_CHF",
        "AUD_USD", "NZD_USD",
        "USD_JPY", "EUR_JPY"
    ]

    # Run the monthly backtest
    arbitrage_results = run_triangular_arbitrage_backtest(
        pairs=forex_pairs,
        granularity="M1",   # Monthly candles
        count=60            # Last 5 years (~60 months)
    )

📥 Fetching monthly historical data for:
  - EUR_USD
  - GBP_USD
  - GBP_CHF
  - AUD_USD
  - NZD_USD
  - USD_JPY
  - EUR_JPY

📊 Merged Monthly DataFrame Sample:
                           EUR_USD  GBP_USD  GBP_CHF  AUD_USD  NZD_USD  \
datetime                                                                 
2025-06-16 08:49:00+00:00  1.15792  1.35787  1.10216  0.65108  0.60354   
2025-06-16 08:50:00+00:00  1.15804  1.35810  1.10220  0.65130  0.60372   
2025-06-16 08:51:00+00:00  1.15824  1.35817  1.10222  0.65131  0.60385   
2025-06-16 08:52:00+00:00  1.15826  1.35819  1.10233  0.65145  0.60398   
2025-06-16 08:53:00+00:00  1.15821  1.35800  1.10225  0.65148  0.60398   

                           USD_JPY  EUR_JPY  
datetime                                     
2025-06-16 08:49:00+00:00  144.162  166.926  
2025-06-16 08:50:00+00:00  144.109  166.881  
2025-06-16 08:51:00+00:00  144.104  166.907  
2025-06-16 08:52:00+00:00  144.104  166.908  
2025-06-16 08:53:00+00:00  144.124  166.926  

In [41]:
# Import libraries
import pandas as pd
import numpy as np
from itertools import permutations
from oandapyV20 import API
from oandapyV20.endpoints.instruments import InstrumentsCandles

# ===== YOUR CREDENTIALS =====
ACCESS_TOKEN = "32a7ce5d62167ef2dda800802386d48a-496bae9b01506113e0a75dbac8e7a693"

# 1. Function to fetch historical data for one instrument
def fetch_forex_data(instrument="EUR_USD", granularity="D", count=5000):
    client = API(access_token=ACCESS_TOKEN, environment="practice")
    params = {"granularity": granularity, "count": count}
    r = InstrumentsCandles(instrument=instrument, params=params)

    try:
        client.request(r)
    except Exception as e:
        raise ConnectionError(f"Failed to fetch data for {instrument}: {e}")

    data = []
    for candle in r.response['candles']:
        data.append({
            'datetime': candle['time'],
            'close': float(candle['mid']['c'])
        })

    df = pd.DataFrame(data)
    df['datetime'] = pd.to_datetime(df['datetime'])
    df.set_index('datetime', inplace=True)
    return df[['close']].rename(columns={'close': instrument})

# 2. Generate all possible triangular triplets from a list of currencies
def generate_all_triangles(currencies):
    # Avoid repeating same triplet in different orders
    seen = set()
    triangles = []

    for c1, c2, c3 in permutations(currencies, 3):
        key = tuple(sorted([c1, c2, c3]))
        if key not in seen:
            seen.add(key)
            triangles.append([c1, c2, c3])
    return triangles

# 3. Detect arbitrage opportunity at a single timestamp
def detect_opportunity(triangle, row):
    c1, c2, c3 = triangle
    p1 = f"{c1}_{c2}"
    p2 = f"{c2}_{c3}"
    p3 = f"{c3}_{c1}"

    try:
        val = 1.0
        val *= row[p1]
        val *= row[p2]
        val *= row[p3]

        profit = val - 1.0
        return profit
    except KeyError:
        return None

# 4. Main backtest function
def run_full_triangular_arbitrage(pairs, granularities=["M1", "W", "D"], count=5000):
    results = []

    print("📊 Fetching data for:", pairs)
    dfs = [fetch_forex_data(pair, granularity="D", count=count) for pair in pairs]
    merged = pd.concat(dfs, axis=1).dropna().sort_index()

    # Extract unique base currencies
    currencies = set()
    for pair in pairs:
        parts = pair.split("_")
        currencies.update(parts)

    triangles = generate_all_triangles(list(currencies))
    print(f"\n🔄 Testing {len(triangles)} possible triangles...")

    for idx, row in merged.iterrows():
        for triangle in triangles:
            profit = detect_opportunity(triangle, row)
            if profit is not None and profit > 0.0001:
                print(f"✅ Opportunity at {idx} | Triangle: {triangle} | Profit: {profit*100:.4f}%")
                results.append({
                    "timestamp": idx,
                    "triangle": str(triangle),
                    "profit_percent": round(profit * 100, 4),
                    "granularity": "D"
                })

    result_df = pd.DataFrame(results)
    print("\n📈 Summary of Found Opportunities:")
    if not result_df.empty:
        print(result_df.to_string(index=False))
    else:
        print("❌ No arbitrage opportunities found.")
    return result_df

# ===== RUN FULL BACKTEST =====
if __name__ == "__main__":
    # List of forex pairs to analyze
    forex_pairs = [
        "EUR_USD", "GBP_USD", "GBP_CHF",
        "AUD_USD", "NZD_USD",
        "USD_JPY", "EUR_JPY"
    ]

    # Run full analysis
    arbitrage_results = run_full_triangular_arbitrage(
        pairs=forex_pairs,
        granularities=["M1", "W", "D"],  # You can add H4, H1 later
        count=5000
    )

📊 Fetching data for: ['EUR_USD', 'GBP_USD', 'GBP_CHF', 'AUD_USD', 'NZD_USD', 'USD_JPY', 'EUR_JPY']

🔄 Testing 35 possible triangles...

📈 Summary of Found Opportunities:
❌ No arbitrage opportunities found.


In [71]:
import ccxt
import pandas as pd
import numpy as np

# 1. Connect to Binance
exchange = ccxt.binance()

# 2. Define triangle
triangle = ['BTC', 'USDT', 'ETH']

# 3. Get price for a pair
def get_price(symbol):
    ticker = exchange.fetch_ticker(symbol)
    return ticker['last']

# 4. Detect arbitrage opportunity
def detect_crypto_arbitrage(triangle):
    c1, c2, c3 = triangle

    p1 = f"{c1}/{c2}"   # e.g., BTC/USDT
    p2 = f"{c2}/{c3}"   # e.g., USDT/ETH
    p3 = f"{c3}/{c1}"   # e.g., ETH/BTC

    try:
        price1 = get_price(p1)
        price2 = get_price(p2)
        price3 = get_price(p3)

        # Start with 1 of asset c1
        amount = 1.0
        amount = amount * price1  # c1 -> c2
        amount = amount * price2  # c2 -> c3
        amount = amount * price3  # c3 -> c1

        profit = amount - 1.0

        print(f"\n💱 Triangle: {triangle}")
        print(f"📊 Prices: {p1}={price1:.8f}, {p2}={price2:.8f}, {p3}={price3:.8f}")
        if profit > 0:
            print(f"✅ Profit: {profit*100:.4f}%")
        else:
            print(f"❌ Loss or no profit: {profit*100:.4f}%")

        return {
            "triangle": str(triangle),
            "profit_percent": round(profit * 100, 4),
            "prices": f"{p1}:{price1}, {p2}:{price2}, {p3}:{price3}"
        }

    except Exception as e:
        print("⚠️ Error fetching data:", e)
        return None

# ===== RUN SCAN =====
if __name__ == "__main__":
    print("🔍 Scanning for crypto triangular arbitrage opportunities...\n")
    result = detect_crypto_arbitrage(["BTC", "USDT", "ETH"])

    print("\n📈 Final Result:")
    if result and result["profit_percent"] > 0:
        print("✅ Opportunity Found!")
        print(result)
    else:
        print("❌ No arbitrage opportunity found at the moment.")

🔍 Scanning for crypto triangular arbitrage opportunities...

⚠️ Error fetching data: binance does not have market symbol USDT/ETH

📈 Final Result:
❌ No arbitrage opportunity found at the moment.


In [73]:
import ccxt
import pandas as pd
import numpy as np

# 1. Connect to Binance
exchange = ccxt.binance()

# 2. Define triangle and valid pairs
triangle = ['BTC', 'USDT', 'ETH']
valid_pairs = {
    'BTC_USDT': 'BTC/USDT',
    'ETH_USDT': 'ETH/USDT',
    'ETH_BTC': 'ETH/BTC'
}

# 3. Get price for a pair
def get_price(symbol):
    ticker = exchange.fetch_ticker(symbol)
    return ticker['last']

# 4. Detect arbitrage opportunity
def detect_crypto_arbitrage(triangle):
    c1, c2, c3 = triangle

    p1 = valid_pairs['BTC_USDT']   # BTC -> USDT
    p2 = valid_pairs['ETH_USDT']   # USDT -> ETH (buy ETH with USDT)
    p3 = valid_pairs['ETH_BTC']    # ETH -> BTC

    try:
        price1 = get_price(p1)     # How much USDT per BTC
        price2 = get_price(p2)     # How much ETH per USDT
        price3 = get_price(p3)     # How much BTC per ETH

        # Start with 1 BTC
        amount = 1.0
        amount *= price1           # BTC -> USDT
        amount *= price2           # USDT -> ETH
        amount *= price3           # ETH -> BTC

        profit = amount - 1.0

        print(f"\n💱 Triangle: {triangle}")
        print(f"📊 Prices: {p1}={price1:.8f}, {p2}={price2:.8f}, {p3}={price3:.8f}")
        if profit > 0:
            print(f"✅ Profit: {profit*100:.4f}%")
        else:
            print(f"❌ Loss or no profit: {profit*100:.4f}%")

        return {
            "timestamp": pd.Timestamp.now(),
            "triangle": str(triangle),
            "profit_percent": round(profit * 100, 4),
            "prices": f"{p1}:{price1}, {p2}:{price2}, {p3}:{price3}"
        }

    except Exception as e:
        print("⚠️ Error fetching data:", e)
        return None

# ===== RUN SCAN =====
if __name__ == "__main__":
    print("🔍 Scanning for crypto triangular arbitrage opportunities...\n")
    result = detect_crypto_arbitrage(triangle)

    print("\n📈 Final Result:")
    if result and result["profit_percent"] > 0:
        print("✅ Opportunity Found!")
        print(result)
    else:
        print("❌ No arbitrage opportunity found at the moment.")

🔍 Scanning for crypto triangular arbitrage opportunities...


💱 Triangle: ['BTC', 'USDT', 'ETH']
📊 Prices: BTC/USDT=106734.62000000, ETH/USDT=2612.89000000, ETH/BTC=0.02448000
✅ Profit: 682712390.4244%

📈 Final Result:
✅ Opportunity Found!
{'timestamp': Timestamp('2025-06-16 12:00:35.198873'), 'triangle': "['BTC', 'USDT', 'ETH']", 'profit_percent': 682712390.4244, 'prices': 'BTC/USDT:106734.62, ETH/USDT:2612.89, ETH/BTC:0.02448'}


In [75]:
import ccxt

# 1. Connect to Binance
exchange = ccxt.binance()

# 2. Define triangle
triangle = ['BTC', 'USDT', 'ETH']

# 3. Get price for a pair
def get_price(symbol):
    ticker = exchange.fetch_ticker(symbol)
    return ticker['last']

# 4. Detect arbitrage opportunity
def detect_crypto_arbitrage(triangle):
    c1, c2, c3 = triangle

    price_btc_usdt = get_price('BTC/USDT')  # BTC to USDT
    price_eth_usdt = get_price('ETH/USDT')  # ETH to USDT
    price_eth_btc = get_price('ETH/BTC')    # ETH to BTC

    print(f"\n💱 Triangle: {triangle}")
    print(f"📊 Prices: BTC/USDT={price_btc_usdt:.2f}, ETH/USDT={price_eth_usdt:.2f}, ETH/BTC={price_eth_btc:.8f}")

    # Start with 1 BTC
    start_btc = 1.0
    usdt_amount = start_btc * price_btc_usdt  # BTC -> USDT
    eth_amount = usdt_amount / price_eth_usdt  # USDT -> ETH
    final_btc = eth_amount * price_eth_btc     # ETH -> BTC

    profit = final_btc - start_btc

    if profit > 0:
        print(f"✅ Profit: {profit*100:.4f}%")
    else:
        print(f"❌ Loss or no profit: {profit*100:.4f}%")

    return {
        "start_btc": start_btc,
        "final_btc": final_btc,
        "profit_percent": round(profit * 100, 4),
        "prices": f"BTC/USDT:{price_btc_usdt}, ETH/USDT:{price_eth_usdt}, ETH/BTC:{price_eth_btc}"
    }

# ===== RUN SCAN =====
if __name__ == "__main__":
    print("🔍 Scanning for crypto triangular arbitrage opportunities...\n")
    result = detect_crypto_arbitrage(triangle)

    print("\n📈 Final Result:")
    if result and result["profit_percent"] > 0:
        print("✅ Opportunity Found!")
        print(result)
    else:
        print("❌ No arbitrage opportunity found at the moment.")

🔍 Scanning for crypto triangular arbitrage opportunities...


💱 Triangle: ['BTC', 'USDT', 'ETH']
📊 Prices: BTC/USDT=106788.33, ETH/USDT=2611.42, ETH/BTC=0.02445000
❌ Loss or no profit: -0.0171%

📈 Final Result:
❌ No arbitrage opportunity found at the moment.


In [77]:
import ccxt
import time
from itertools import permutations

# ===== CONFIGURATION =====
exchange = ccxt.binance()
coins = ['BTC', 'USDT', 'ETH', 'BNB', 'XRP', 'DOGE', 'ADA']  # Add more if needed
min_profit_threshold = 0.2  # Only show opportunities with at least this % profit

# ===== HELPER FUNCTIONS =====
def get_price(pair):
    try:
        return exchange.fetch_ticker(pair)['last']
    except Exception as e:
        # print(f"⚠️ Error fetching {pair}: {e}")
        return None

def get_valid_pairs(triangle):
    """Returns valid trading pairs for a given triangle if they exist"""
    c1, c2, c3 = triangle

    pair1 = f"{c1}/{c2}"
    pair2 = f"{c2}/{c3}"
    pair3 = f"{c3}/{c1}"

    price1 = get_price(pair1)
    price2 = get_price(pair2)
    price3 = get_price(pair3)

    if None in [price1, price2, price3]:
        return None

    return {
        "pair1": pair1,
        "pair2": pair2,
        "pair3": pair3,
        "price1": price1,
        "price2": price2,
        "price3": price3
    }

def detect_opportunity(triangle):
    prices = get_valid_pairs(triangle)
    if not prices:
        return None

    c1, c2, c3 = triangle
    p1, p2, p3 = prices["pair1"], prices["pair2"], prices["pair3"]
    price1, price2, price3 = prices["price1"], prices["price2"], prices["price3"]

    # Start with 1 of c1
    amount = 1.0
    amount *= price1  # c1 -> c2
    amount *= price2  # c2 -> c3
    amount *= price3  # c3 -> c1

    profit_percent = (amount - 1) * 100

    if profit_percent > min_profit_threshold:
        return {
            "timestamp": pd.Timestamp.now(),
            "triangle": str(triangle),
            "profit_percent": round(profit_percent, 4),
            "prices": f"{p1}:{price1:.8f}, {p2}:{price2:.8f}, {p3}:{price3:.8f}"
        }
    return None

# ===== MAIN SCANNER =====
if __name__ == "__main__":
    print("🔍 Starting multi-crypto triangular arbitrage scanner...\n")
    print(f"Supported coins: {coins}")
    print(f"Minimum profit threshold: {min_profit_threshold}%\n")

    found_any = False

    # Generate all possible 3-coin triangles
    for triangle in permutations(coins, 3):
        result = detect_opportunity(triangle)
        if result:
            print(f"✅ Opportunity Found: {result['triangle']}")
            print(f"📊 Profit: {result['profit_percent']}%")
            print(f"💱 Prices: {result['prices']}\n")
            found_any = True

    if not found_any:
        print("❌ No arbitrage opportunities found at the moment.")
    else:
        print("🎉 Done scanning — check above for profitable triangles.")

🔍 Starting multi-crypto triangular arbitrage scanner...

Supported coins: ['BTC', 'USDT', 'ETH', 'BNB', 'XRP', 'DOGE', 'ADA']
Minimum profit threshold: 0.2%

❌ No arbitrage opportunities found at the moment.


In [79]:
import ccxt
from itertools import permutations

# ===== CONFIGURATION =====
exchange = ccxt.binance()

# Expanded list of tradable coins
coins = [
    'BTC', 'USDT', 'ETH', 'BNB', 'XRP', 'DOGE', 'ADA',
    'SOL', 'DOT', 'SHIB', 'MATIC', 'LTC', 'LINK', 'ATOM', 'XMR'
]

min_profit_threshold = 0.2  # Only show opportunities above this %

# ===== HELPER FUNCTIONS =====
def get_price(pair):
    try:
        return exchange.fetch_ticker(pair)['last']
    except Exception as e:
        return None

def detect_opportunity(triangle):
    c1, c2, c3 = triangle

    pair1 = f"{c1}/{c2}"
    pair2 = f"{c2}/{c3}"
    pair3 = f"{c3}/{c1}"

    price1 = get_price(pair1)
    price2 = get_price(pair2)
    price3 = get_price(pair3)

    if None in [price1, price2, price3]:
        return None

    # Start with 1 of c1
    amount = 1.0
    amount *= price1  # c1 -> c2
    amount *= price2  # c2 -> c3
    amount *= price3  # c3 -> c1

    profit_percent = (amount - 1) * 100

    if profit_percent > min_profit_threshold:
        return {
            "triangle": str(triangle),
            "profit_percent": round(profit_percent, 4),
            "prices": f"{pair1}:{price1:.8f}, {pair2}:{price2:.8f}, {pair3}:{price3:.8f}"
        }
    return None

# ===== MAIN SCANNER =====
if __name__ == "__main__":
    print("🔍 Starting expanded crypto triangular arbitrage scanner...\n")
    print(f"Supported coins: {coins}")
    print(f"Minimum profit threshold: {min_profit_threshold}%\n")

    found_any = False

    # Generate all possible 3-coin triangles
    for triangle in permutations(coins, 3):
        # Skip if same coin used twice
        if len(set(triangle)) < 3:
            continue

        result = detect_opportunity(triangle)
        if result:
            print(f"✅ Opportunity Found: {result['triangle']}")
            print(f"📊 Profit: {result['profit_percent']}%")
            print(f"💱 Prices: {result['prices']}\n")
            found_any = True

    if not found_any:
        print("❌ No arbitrage opportunities found at the moment.")
    else:
        print("🎉 Done scanning — check above for profitable triangles.")

🔍 Starting expanded crypto triangular arbitrage scanner...

Supported coins: ['BTC', 'USDT', 'ETH', 'BNB', 'XRP', 'DOGE', 'ADA', 'SOL', 'DOT', 'SHIB', 'MATIC', 'LTC', 'LINK', 'ATOM', 'XMR']
Minimum profit threshold: 0.2%

❌ No arbitrage opportunities found at the moment.
